In [6]:
import tensorflow as tf
import numpy as np 
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,GlobalMaxPooling2D,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

In [17]:
import os
import shutil

source_dirs = ['databrain/Training', 'databrain/Testing']
target_dir = 'databrain_combined'

os.makedirs(target_dir, exist_ok=True)

for src in source_dirs:
    for class_name in os.listdir(src):
        class_src = os.path.join(src, class_name)
        class_target = os.path.join(target_dir, class_name)
        os.makedirs(class_target, exist_ok=True)

        for file in os.listdir(class_src):
            file_src = os.path.join(class_src, file)
            file_target = os.path.join(class_target, file)
            shutil.copy(file_src, file_target)


In [18]:
IMG_SIZE = (224,224)
BATCH_SIZE = 32
EPOCHS = 15
databese = 'databrain_combined'

generat = ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.2,
    rotation_range = 20,
    zoom_range = 0.15,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    shear_range = 0.1,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

train_data = generat.flow_from_directory(
    databese,
    target_size = IMG_SIZE,
    batch_size = BATCH_SIZE,
    class_mode = 'categorical',
    subset = 'training'
)

valid_data = generat.flow_from_directory(
    databese,
    target_size = IMG_SIZE,
    batch_size = BATCH_SIZE,
    class_mode = 'categorical',
    subset = 'validation'
)

class_name = list(train_data.class_indices.keys())
print('classes:', class_name)

Found 5619 images belonging to 4 classes.
Found 1404 images belonging to 4 classes.
classes: ['glioma', 'meningioma', 'notumor', 'pituitary']


In [22]:
BRAIN_MODEL = MobileNetV2(
    weights='imagenet',
    include_top = False,
    input_shape = (224,224,3)
)
BRAIN_MODEL.trainable = False

X = BRAIN_MODEL.output
X = GlobalMaxPooling2D()(X)
X = Dropout(0.3)(X)
X = Dense(128, activation='relu')(X)
X = Dropout(0.2)(X)
output = Dense(4, activation='softmax')(X)

BRAIN_MODEL = Model(
    inputs = BRAIN_MODEL.input,
    outputs = output
)
optimizer = Adam(learning_rate=1e-4)
BRAIN_MODEL.compile(
    optimizer = optimizer,
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)


In [20]:
BRAIN_MODEL.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer_2[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,422,210 (9.24 MB)

 Trainable params: 164,226 (641.51 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [23]:
callback = [
    EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights = True
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        patience=2,
        factor=0.2
    ),
    ModelCheckpoint(
        'Brain_tumor_model.keras',
        save_best_only = True
    )
]

In [24]:
store_info = BRAIN_MODEL.fit(
    train_data,
    validation_data = valid_data,
    epochs = EPOCHS,
    callbacks = callback
)

Epoch 1/15
176/176 ━━━━━━━━━━━━━━━━━━━━ 251s 1s/step - accuracy: 0.4252 - loss: 3.0717 - val_accuracy: 0.7372 - val_loss: 0.7055 - learning_rate: 1.0000e-04
Epoch 2/15
176/176 ━━━━━━━━━━━━━━━━━━━━ 247s 1s/step - accuracy: 0.6639 - loss: 1.0611 - val_accuracy: 0.7607 - val_loss: 0.6440 - learning_rate: 1.0000e-04
Epoch 3/15
176/176 ━━━━━━━━━━━━━━━━━━━━ 250s 1s/step - accuracy: 0.7046 - loss: 0.8107 - val_accuracy: 0.7821 - val_loss: 0.5985 - learning_rate: 1.0000e-04
Epoch 4/15
176/176 ━━━━━━━━━━━━━━━━━━━━ 291s 2s/step - accuracy: 0.7472 - loss: 0.6521 - val_accuracy: 0.8013 - val_loss: 0.5585 - learning_rate: 1.0000e-04
Epoch 5/15
176/176 ━━━━━━━━━━━━━━━━━━━━ 277s 2s/step - accuracy: 0.7628 - loss: 0.6155 - val_accuracy: 0.8120 - val_loss: 0.5291 - learning_rate: 1.0000e-04
Epoch 6/15
176/176 ━━━━━━━━━━━━━━━━━━━━ 263s 1s/step - accuracy: 0.7843 - loss: 0.5544 - val_accuracy: 0.8120 - val_loss: 0.4863 - learning_rate: 1.0000e-04
Epoch 7/15
176/176 ━━━━━━━━━━━━━━━━━━━━ 255s 1s/step - acc

In [25]:
# Fine-tuning (AutoTUNE)
print("\n Début du Fine-Tuning des couches profondes...\n")
BRAIN_MODEL.trainable = True
BRAIN_MODEL.compile(
    optimizer = Adam(learning_rate=1e-5),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)
fine_tune_epochs = 10
total_epochs = EPOCHS + fine_tune_epochs
BRAIN_MODEL.fit(
    train_data,
    validation_data = valid_data,
    epochs = total_epochs,
    initial_epoch = store_info.epoch[-1],
    callbacks = callback
)

print("\n Évaluation finale du modèle :")


 Début du Fine-Tuning des couches profondes...

Epoch 15/25
176/176 ━━━━━━━━━━━━━━━━━━━━ 255s 1s/step - accuracy: 0.8310 - loss: 0.4174 - val_accuracy: 0.8412 - val_loss: 0.4104 - learning_rate: 1.0000e-05
Epoch 16/25
176/176 ━━━━━━━━━━━━━━━━━━━━ 231s 1s/step - accuracy: 0.8368 - loss: 0.4320 - val_accuracy: 0.8390 - val_loss: 0.3892 - learning_rate: 1.0000e-05
Epoch 17/25
176/176 ━━━━━━━━━━━━━━━━━━━━ 265s 2s/step - accuracy: 0.8335 - loss: 0.4241 - val_accuracy: 0.8383 - val_loss: 0.4076 - learning_rate: 1.0000e-05
Epoch 18/25
176/176 ━━━━━━━━━━━━━━━━━━━━ 274s 2s/step - accuracy: 0.8244 - loss: 0.4410 - val_accuracy: 0.8412 - val_loss: 0.4058 - learning_rate: 1.0000e-05
Epoch 19/25
176/176 ━━━━━━━━━━━━━━━━━━━━ 276s 2s/step - accuracy: 0.8429 - loss: 0.4279 - val_accuracy: 0.8355 - val_loss: 0.4285 - learning_rate: 2.0000e-06

 Évaluation finale du modèle :


In [27]:
valid_data.reset()
Y_pred = BRAIN_MODEL.predict(valid_data, verbose = 1)
y_pred = np.argmax(Y_pred, axis =1)
y_true = valid_data.classes

print(classification_report(y_true, y_pred, target_names=class_name))
print("\nConfusion Matrix :\n", confusion_matrix(y_true, y_pred))

BRAIN_MODEL.save("Brain_tumor_final.keras")
print("\nModèle sauvegardé sous brain_tumor_final.keras")

44/44 ━━━━━━━━━━━━━━━━━━━━ 49s 1s/step
              precision    recall  f1-score   support

      glioma       0.29      0.25      0.27       324
  meningioma       0.21      0.19      0.20       329
     notumor       0.28      0.30      0.29       400
   pituitary       0.28      0.32      0.30       351

    accuracy                           0.27      1404
   macro avg       0.26      0.26      0.26      1404
weighted avg       0.27      0.27      0.27      1404


Confusion Matrix :
 [[ 81  77  91  75]
 [ 63  62 114  90]
 [ 70  89 119 122]
 [ 66  70 102 113]]

Modèle sauvegardé sous brain_tumor_final.keras
